In [1]:
import dotenv
dotenv.load_dotenv('.env', verbose=True, override=True)
import os
from langchain_openai import AzureChatOpenAI
COMPLETION_TOKENS = 2500
llm = AzureChatOpenAI(deployment_name=os.environ["GPT_DEPLOYMENT"], temperature=0.5, max_tokens=COMPLETION_TOKENS)

In [3]:
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dotenv.load_dotenv('.env', verbose=True, override=True)

from common.customAzureSearch import CustomAzureSearchRetriever
from azure.identity import DefaultAzureCredential

DOCSEARCH_PROMPT_TEXT = """Answer the question thoroughly, based **ONLY** on the following context:
{context}

Question: {question}
"""

DOCSEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("system", DOCSEARCH_PROMPT_TEXT),
        MessagesPlaceholder(variable_name="history", optional=True),
        ("human", "{question}"),
    ]
)

cred = DefaultAzureCredential()
retriever = CustomAzureSearchRetriever(indexes=[os.environ['INDEX_NAME']], credential=cred, topK=5, reranker_threshold=1)


In [5]:
chain = (
    {
        "context": itemgetter("question") | retriever, # Passes the question to the retriever and the results are assign to context
        "question": itemgetter("question")
    }
    | DOCSEARCH_PROMPT
)
resp = chain.invoke({"question": "Which fillaments does Longer recommend?"})
print(resp)

messages=[SystemMessage(content='Answer the question thoroughly, based **ONLY** on the following context:\n[Document(page_content=\'the printing parameters according to\\n\\nindividual needs. We recommend using the default parameters and\\n\\nLONGER filaments to get the best printing results, please refer to\\n\\npage 11 to import the default settings.\\n\\n\\n\\nwww.longer3d.com support@longer3d.com22\\n\\n\\n\\nwww.longer3d.com support@longer3d.com23\\n\\n(4)Slice and save Gcode files\\n\\nClick Slice to generate G-code for printing.\\n\\nSave to removable drive (Save the gcode file to storage card). And\\n\\nthe software will evaluate the time required to print the file and the\\n\\nweight of filament.\\n\\nNOTE: “Save to removable drive” option will apear when microSD\\n\\ncard is inserted.\\n\\n(5)More\\n\\nA) Slice and preview\\n\\nClick the "Slice" button in the lower right corner of the software.\\n\\nAfter the slicing is finished, click the "preview" button to preview the\\n\\